# 文本预处理

一篇文章可以看作字符或单词的序列，常见文本数据的预处理一般包括4个步骤：
1. 读入文本
2. 分词
3. 建立字典，将单词映射成索引
4. 将文本从词的序列转为索引的序列

以下以英文小说[Time Machine](http://www.gutenberg.org/ebooks/35)为例，英文。

In [1]:
import collections
import re
import nltk

## 读入文本

In [2]:
def read_time_machine():
    with open("timemachine.txt","r",encoding="utf-8") as f:
        # 仅保留a-zA-Z的字符
        lines = [re.sub('[^a-z]+',' ',line.strip().lower()) for line in f]
    return lines

In [3]:
lines = read_time_machine()
len(lines)

3702

## 分词
字-**词**-句-**文章**

**第一层列表并不是句子，因为输入并不是一个句子一行。**

In [4]:
def tokenize(sentences, token="word"):
    if token == "word":
        return [sentence.split(" ") for sentence in sentences 
                                        if sentence.strip() != "" and 
                                           len(sentence.split(" "))>1]
    elif token == "char":
        return [list(sentence) for sentence in sentences]
    else:
        print("Error：unknown token type"+token)        

In [5]:
tokens = tokenize(lines)
tokens[0]

['the', 'time', 'machine', 'by', 'h', 'erbert', 'g', 'eorge', 'wells', '']

## 建立字典

In [6]:
def count_corpus(sentences):
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)  # 返回一个字典，记录每个词的出现次数

In [7]:
len(count_corpus(tokens))

4932

In [8]:
class Vocab():
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = count_corpus(tokens)
        self.token_freqs = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['', '', '', '']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        self.idx_to_token += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

In [9]:
vocab = Vocab(tokens)

In [10]:
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('erbert', 6), ('g', 7), ('eorge', 8), ('wells', 9)]


In [11]:
vocab[["the","time"]]

[1, 2]

In [12]:
vocab.to_tokens([2,3])

['time', 'machine']

## 将词转为索引

In [13]:
for i in range(0, 10):
    print("line {} :".format(i),len(tokens[i]))
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

line 0 : 10
words: ['the', 'time', 'machine', 'by', 'h', 'erbert', 'g', 'eorge', 'wells', '']
indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
line 1 : 12
words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of']
indices: [1, 2, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
line 2 : 11
words: ['him', 'was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes']
indices: [20, 21, 22, 23, 24, 25, 17, 26, 27, 28, 29]
line 3 : 11
words: ['shone', 'and', 'twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and']
indices: [30, 31, 32, 31, 27, 33, 34, 35, 21, 36, 31]
line 4 : 11
words: ['animated', 'the', 'fire', 'burned', 'brightly', 'and', 'the', 'soft', 'radiance', 'of', 'the']
indices: [37, 1, 38, 39, 40, 31, 1, 41, 42, 19, 1]
line 5 : 10
words: ['incandescent', 'lights', 'in', 'the', 'lilies', 'of', 'silver', 'caught', 'the', 'bubbles']
indices: [43, 44, 45, 1, 46, 19, 47, 48, 1, 49]
line 6 : 11
words: ['that', 'flas

## 用现有工具进行分词
增加的功能
- 标点符号的语义信息
- doesn't等缩略词的处理

In [14]:
text = "Mr. Chen doesn't agree with my suggestion."

### spacy

In [15]:
import spacy
# python -m spacy download en_core_web_sm

In [16]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [17]:
print([token.text for token in doc])

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


### nltk

In [18]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import data

In [19]:
print(word_tokenize(text))

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


# 语言模型

一段自然语言文本可以看作是一个离散时间序列，给定一个长度为T的词的序列，语言模型的目标是评估该序列是否合理，即计算该序列的概率。

- 语料库：corpus

语言模型的参数是**词的概率**以及**给定前n个词的情况下的条件概率**

## n元语法（n-gram）

n元语法通过马尔可夫假设简化模型，马尔可夫假设是指一个词的出现只与前面(n-1个相关，即n阶马尔科夫链。

语言模型的n阶马尔可夫链，即n-gram。

$$
P(w_1, w_2, \ldots, w_T) \approx \prod_{t=1}^T P(w_t \mid w_{t-(n-1)}, \ldots, w_{t-1}) .
$$

【常用的n-gram模型】
- unigram：(2)
- bigram：(3)
- trigram

$$\begin{split}\begin{aligned}
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2) P(w_3) P(w_4) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_2) P(w_4 \mid w_3) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_2, w_3) .
\end{aligned}\end{split}$$

【n-gram缺陷】
1. 参数空间过大
2. 数据过于稀疏(齐夫定律）

## 周杰伦歌词数据集
从[d2lzh的github](https://github.com/d2l-ai/d2l-zh/blob/master/data/jaychou_lyrics.txt.zip)上下载。

In [20]:
import zipfile
with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')

In [21]:
print(len(corpus_chars))
print(corpus_chars[: 40])
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[: 10000]

63282
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


### 建立字符索引

In [22]:
idx_to_char = list(set(corpus_chars))
char_to_idx = {char:i for i,char in enumerate(idx_to_char)}
vocab_size = len(char_to_idx)

In [23]:
vocab_size

1027

In [24]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]  # 将每个字符转化为索引，得到一个索引的序列
sample = corpus_indices[: 20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [755, 804, 909, 656, 901, 787, 192, 755, 804, 854, 127, 11, 793, 853, 506, 532, 192, 755, 804, 854]


In [25]:
def load_data_jay_lyrics():
    with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

## 时序数据的采样
时间序列有大量重复的样本，因此需要采样。

序列长度为T，时间步数为n，共有$T-n$个合法样本。

- 随机采样
- 相邻采样

**采样的的结果是（批量大小，时间步数）**

### 随机采样

下面的代码每次从数据里随机采样一个小批量。其中批量大小`batch_size`是每个小批量的样本数，`num_steps`是每个样本所包含的时间步数。
在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。

**硬性分成了多个片段，片段之间的联系破坏了。**

In [26]:
import random
import torch

In [27]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    num_examples = (len(corpus_indices)-1)//num_steps
    # example_indices记录每个分组的第一个下标
    example_indices = [i*num_steps for i in range(num_examples)]
    random.shuffle(example_indices)
    
    def _data(i):
        return corpus_indices[i:i+num_steps]
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
    for i in range(0,num_examples, batch_size):
        batch_indices = example_indices[i:i+batch_size]
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)
        

In [28]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [18, 19, 20, 21, 22, 23]], device='cuda:0') 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [19, 20, 21, 22, 23, 24]], device='cuda:0') 

X:  tensor([[12, 13, 14, 15, 16, 17],
        [ 6,  7,  8,  9, 10, 11]], device='cuda:0') 
Y: tensor([[13, 14, 15, 16, 17, 18],
        [ 7,  8,  9, 10, 11, 12]], device='cuda:0') 



### 相邻采样

在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。

也把corpus序列分成了多个片段。

In [29]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [30]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]], device='cuda:0') 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]], device='cuda:0') 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]], device='cuda:0') 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]], device='cuda:0') 



# 循环神经网络基础

目的：基于当前的输入与过去的输入序列，预测序列的下一个字符。

给定样本数为 n 、输入个数（特征数或特征向量维度）为 d 的小批量数据样本
$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$。

## RNN基本概念
### 隐藏层
$$
\text{H}_t = \phi(\text{X}_t \text{W}_{xh} + \text{H}_{t-1} \text{W}_{hh}  + \text{b}_h).
$$
其中，$\text{W}_{xh} \in \mathbb{R}^{d \times h}$，$\text{W}_{hh} \in \mathbb{R}^{h \times h}$，$\text{b}_{h} \in \mathbb{R}^{1 \times h}$，$\phi$函数是非线性激活函数
### 输出层
在时间步$t$，输出层的输出为：
$$
\text{O}_t = \text{H}_t \text{W}_{hq} + \text{b}_q.
$$

- n：sample_size
- d：onehot_size
- h：hidden_units 

## 从零开始实现RNN

语料使用周杰伦歌词。中文字符级的RNN语言模型。

In [31]:
import torch
import torch.nn as nn
import time
import math
print(torch.__version__)
print(torch.cuda.is_available())

1.0.0
True


### 准备数据

In [32]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
vocab_size

1027

### ont-hot向量
```scatter_(dim, index, src) → Tensor```，与gather是反函数，index是LongTensor类型。

- 输入：语言模型的输出（batch,step_size）；
- 输出：RNN的输入（step_size, batch_size, onehot_size）

#### one_hot(...)

In [34]:
def one_hot(x, n_class, dtype=torch.float):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    result.scatter_(1, x.long().view(-1,1), 1)
    return result

In [35]:
x = torch.tensor([0, 2])
x_one_hot = one_hot(x, vocab_size)
print(x_one_hot)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])


In [36]:
x.shape[0],vocab_size

(2, 1027)

#### to_onehot(...)

```语言模型 -> (batch_size, step_size) -> OneHot -> (step_size, batch_size, onehot_size) -> RNN```

In [37]:
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

In [38]:
X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1027])


(2,5) -> (5,2,1027)

### 模型参数初始化

![Image Name](https://cdn.kesci.com/upload/image/q5jkm0v44i.png?imageView2/0/w/640/h/640)

(step_size, batch_size, onehot_size) -> RNN

- 输入与输出形状相同
- 输入的特征数即onehot_size

In [39]:
import torch
from torch import nn
from torch.nn import init

In [40]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

#### get_params()

In [41]:
def get_params():
    def _one(shape):
        param = torch.zeros(shape, device=device, dtype=torch.float32)
        init.normal_(param, 0, 0.01)
        return nn.Parameter(param)
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device))
    W_xh.requires_grad_(requires_grad=True)
    W_hh.requires_grad_(requires_grad=True)
    b_h.requires_grad_(requires_grad=True)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
    W_hq.requires_grad_(requires_grad=True)
    b_q.requires_grad_(requires_grad=True)
    return (W_xh, W_hh, b_h, W_hq, b_q)

### 定义模型
仅预测单步。

#### rnn(...)

In [42]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$

init_state是batch_size x num_hiddens

#### init_rnn_state(...)

In [43]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

#### 小测试

In [44]:
X

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [45]:
X.shape, num_hiddens, vocab_size

(torch.Size([2, 5]), 256, 1027)

In [46]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)

In [47]:
print(len(inputs), inputs[0].shape)
print(len(outputs), outputs[0].shape)
print(len(state), state[0].shape)
print(len(state_new), state_new[0].shape)

5 torch.Size([2, 1027])
5 torch.Size([2, 1027])
1 torch.Size([2, 256])
1 torch.Size([2, 256])


In [48]:
outputs2, state_new2 = rnn(outputs, state_new, params)

In [49]:
print(len(outputs2), outputs2[0].shape)
print(len(state_new2), state_new2[0].shape)

5 torch.Size([2, 1027])
1 torch.Size([2, 256])


### 裁剪梯度
RNN容易出现梯度衰减或者梯度爆炸,梯度裁剪可以应对梯度爆炸。

1. 求所有的模型参数梯度的范数
2. 检测范数是否超过阈值
3. 归一化到小于阈值theta

$$
 \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}
$$

#### grad_clipping(...)

In [50]:
def grad_clipping(params, theta, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

### 定义预测函数

- ``prefix```：含有数个字符的字符串；
- ```num_chars```：需要预测的字符个数

In [51]:
idx_to_char[4], char_to_idx["砖"]

('译', 966)

#### predict_rnn(...)

- ```init_rnn_state(batch_size, num_hiddens, device) -> state```
- ```to_onehot(X, n_class)：语言模型(batch_size, step_size) -> RNNin(step_size, batch_size, onehot_size)```
- ```rnn(inputs, state, params) -> outputs, (H,)```

In [52]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]   
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
#         print("X ",len(X),X[0].shape)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
#         print("Y ",len(X),Y[0].shape)
        # prefix范围的字符只改变隐藏层状态
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y[0].argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

- batch_size=1
- step_size=1
- onehot_size=1027

In [53]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开败早喘短逅陷完界产庭'

### 困惑度（评价指标）

困惑度是对交叉熵损失函数做指数运算后得到的值。

- 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
- 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
- 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

任何一个有效的模型困惑度必须小于类别个数。

### 训练模型
- 如使用相邻采样，在epoch开始时初始化隐藏状态，在每个小批量更新前使用detach函数从计算图分离隐藏状态。
- 如使用随机采样，在每个小批量更新前初始化隐藏状态。

【RNN】
- ```input```：num_steps个形状为(batch_size, vocab_size)的矩阵
- ```outputs```：num_steps个形状为(batch_size, vocab_size)的矩阵

In [56]:
import d2lzh as d2l
import numpy as np

In [57]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [58]:
for X, Y in data_iter_random(corpus_indices, batch_size, num_steps, device):
    break

In [59]:
X.shape,Y.shape

(torch.Size([32, 35]), torch.Size([32, 35]))

In [60]:
Y.t().shape

torch.Size([35, 32])

#### 优化函数sgd(...)

In [61]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr*param.grad / batch_size

#### train_and_predict_rnn(...)

In [62]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_siaze, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:
                for s in state:
                    s.detach_()
            inputs = to_onehot(X, vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后outputs形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后变成形状为(num_steps * batch_size,)的向量，这样跟输出的行一一对应
            print()
            y = torch.flatten(Y.t())
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [63]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

RuntimeError: The size of tensor a (29) must match the size of tensor b (32) at non-singleton dimension 0

In [ ]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

## RNN的简洁实现

In [64]:
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)
num_steps, batch_size = 35, 2
X = torch.rand(num_steps, batch_size, vocab_size)
state = None
Y, state_new = rnn_layer(X, state)
print(Y.shape, state_new.shape)

torch.Size([35, 2, 256]) torch.Size([1, 2, 256])


### 定义网络

In [65]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, inputs, state):
        # inputs.shape: (batch_size, num_steps)
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X)  # X.shape: (num_steps, batch_size, vocab_size)
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])  # hiddens.shape: (num_steps * batch_size, hidden_size)
        output = self.dense(hiddens)
        return output, state

### 模型预测

In [66]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

#### 随机权重预测

In [67]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开痛呜橱橱橱橱橱橱橱橱'

In [68]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state[0].detach_()
                    state[1].detach_()
                else: 
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.T)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [69]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

AttributeError: 'torch.device' object has no attribute 'device_typeid'